In [1]:
import pandas as pd
import ast
import json

In [2]:
# df = pd.read_csv('RecipeNLG_dataset.csv')

In [4]:
# df[df['ingredients'].str.contains("elephant")]

,Unnamed: 0,title,ingredients,directions,link,source,NER
4858,4858,Elephant Stew,"[""1 elephant, medium size"", ""2 rabbits (option...","[""Cut up elephant in bite size pieces."", ""This...",www.cookbooks.com/Recipe-Details.aspx?id=362882,Gathered,"[""rabbits"", ""salt""]"
9238,9238,Elephant Stew,"[""1 elephant"", ""brown gravy (leftover or prepa...","[""Cut elephant into bite size pieces."", ""This ...",www.cookbooks.com/Recipe-Details.aspx?id=698049,Gathered,"[""elephant"", ""brown gravy"", ""peas"", ""rabbits"",..."
10870,10870,Elephant Stew,"[""1 large elephant"", ""brown gravy"", ""salt and ...","[""Cut elephant into bite size pieces."", ""This ...",www.cookbooks.com/Recipe-Details.aspx?id=892229,Gathered,"[""elephant"", ""brown gravy"", ""salt"", ""mushrooms..."
15301,15301,Elephant Stew,"[""1 elephant (medium-size)"", ""salt and pepper ...","[""Cut elephant into small bite-size pieces."", ...",www.cookbooks.com/Recipe-Details.aspx?id=932077,Gathered,"[""salt"", ""rabbits"", ""brown gravy""]"
19966,19966,Elephant Stew,"[""1 young elephant"", ""300 lb. potatoes"", ""50 l...","[""Cut elephant in bite-size pieces."", ""Boil fo...",www.cookbooks.com/Recipe-Details.aspx?id=738780,Gathered,"[""potatoes"", ""onions"", ""carrots"", ""season"", ""r..."
...,...,...,...,...,...,...,...
2189148,2189148,Flank Steak With Chimichurri Sauce,"[""1 12 lbs trimmed flank steaks"", ""1 12 teaspo...","[""FOR THE SAUCE: Puree all the Chimichurri ing...",www.food.com/recipe/flank-steak-with-chimichur...,Recipes1M,"[""kosher salt"", ""ground cumin"", ""ground corian..."
2193024,2193024,Broccoli And Angel Hair Pasta Recipe,"[""1 lrg broccoli head"", ""1 lb Angel hair pasta...","[""Cut broccoli into small pcs."", ""Chop elephan...",cookeatshare.com/recipes/broccoli-and-angel-ha...,Recipes1M,"[""hair pasta"", ""Garlic"", ""Margarine""]"
2210420,2210420,Badda Bing Pizza With Pesto-Esque Sauce,"[""1 pizza crust, see Easy Peezy Pizza Dough (B...","[""Prepare the pizza dough as indicated in reci...",www.food.com/recipe/badda-bing-pizza-with-pest...,Recipes1M,"[""pizza crust"", ""onion"", ""mushrooms"", ""garlic""..."
2228274,2228274,Elephant Stew,"[""1 elephant"", ""Brown gravy"", ""2 rabbits (opti...","[""Cut elephant into small bite-sized pieces."",...",online-cookbook.com/goto/cook/rpage/000063,Recipes1M,"[""elephant"", ""Brown gravy"", ""rabbits"", ""Salt""]"


In [3]:
# steps = df['directions'].apply(ast.literal_eval).tolist()

In [18]:
# max = 0
# maxl = None
# count = 0
# for l in steps:
#     slen = len(" ".join(l).split())
#     if slen > 400:
#         count += 1
#     if slen > max:
#         max = slen
#         maxl = l
# print(count)
# print(max)
# print(maxl)

19628
2649
['In a small saucepan, combine the sugar, citric acid, and water and let stand for 30 minutes.', 'Then bring the mixture to a boil over medium heat and cook for 2 to 3 minutes, or until the syrup turns pale yellow (the color of canola oil).', 'If the citric acid turns dark yellow during boiling, swirl the pan or stir the contents to distribute.', 'Remove from the heat and pour into a large measuring cup or heatproof bowl.', 'Set the syrup aside for about 4 hours to cool completely.', 'It will thicken and darken a bit as it cools.', 'Then cover with plastic wrap and keep at room temperature.', 'There should be a scant 1 cup syrup.', '(The syrup can be made up to 3 days in advance and kept at room temperature until using.)', 'Position a rack in the middle of the oven and preheat to 250F.', 'Line a baking sheet with parchment paper.', 'Crack each salted egg and separate the yolk from the white, discarding the white and depositing the yolk on the prepared baking sheet.', 'Discar

In [22]:
# steps400 = [" ".join(l) for l in steps if len(" ".join(l).split()) <= 400]

In [26]:
# with open("steps400w.json", "w") as f:
#     json.dump(steps400[:100000], f)

In [27]:
# with open("steps400wfull.json", "w") as f:
#     json.dump(steps400, f)

In [2]:
df = pd.read_parquet("nlg_refined.gzip")

In [3]:
df = df[df.source == "Gathered"]

In [4]:
ingr = df['ingr'].tolist()

In [13]:
ingr[45245]

array(['1 egg, beaten', '1 stick butter', '1 c. sugar', '1 tsp. vanilla',
       '2 c. self-rising flour'], dtype=object)

In [8]:
def is_frac(string):
    l = string.split("/")
    # print(l)
    try:
        l = [int(w) for w in l]
    except:
        return False
    if len(l) == 1:
        return l[0]
    return round(l[0]/l[1], 2)

In [9]:
ingr_sent = []
new_ingr = []
for l in ingr:
    temp_ingr = []
    for i in l:
        i = i.replace("(", "( ").replace(")", " )")
        temp_split = i.split()
        # print(temp_split)
        for ind in range(len(temp_split)):
            frac = is_frac(temp_split[ind])
            if frac:
                temp_split[ind] = frac
        merge_temp = []
        for ind in range(len(temp_split)):
            try:
                int(temp_split[ind])
                if isinstance(merge_temp[-1], int):
                    merge_temp[-1] += temp_split[ind]
                else:
                    merge_temp.append(temp_split[ind])
            except:
                merge_temp.append(temp_split[ind])
        temp_ingr.append(" ".join([str(w) for w in merge_temp]).replace("( ", "(").replace(" )", ")"))
    new_ingr.append(temp_ingr)

In [11]:
savedf = pd.DataFrame({'ingr':new_ingr})

In [13]:
savedf.to_parquet('./bert/ingredient check/ingr1650k.gzip', compression='gzip')